In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../../src/agents")

In [ ]:
from dotenv import load_dotenv

load_dotenv(override=True)

'InstrumentationKey=d2b57dc4-3311-4a42-8d94-d9ce9cd08002;IngestionEndpoint=https://eastus2-3.in.applicationinsights.azure.com/;LiveEndpoint=https://eastus2.livediagnostics.monitor.azure.com/;ApplicationId=1ddc4ad6-8c77-40af-8e5e-9753be00bb3d'

In [4]:
from azure.monitor.opentelemetry import configure_azure_monitor
connection_string = os.getenv("APPLICATIONINSIGHTS_CONNECTIONSTRING")
configure_azure_monitor(connection_string=connection_string)

In [24]:
from utils.store import CosmosDataStore

store = CosmosDataStore()

result = await store.get_data("cust001", "customer")

In [25]:
result

{'id': 'cust001',
 'partitionKey': 'customer',
 'name': 'Acme Corporation',
 'pricesheet': {'currency': 'USD',
  'updated': '2025-03-01',
  'items': [{'sku': 'SKU_AAA1', 'price': 40},
   {'sku': 'SKU_AAA3', 'price': 55}]},
 '_rid': 'pCwDAJ9Y5L0BAAAAAAAAAA==',
 '_self': 'dbs/pCwDAA==/colls/pCwDAJ9Y5L0=/docs/pCwDAJ9Y5L0BAAAAAAAAAA==/',
 '_etag': '"460144f2-0000-0200-0000-67f3dc960000"',
 '_attachments': 'attachments/',
 '_ts': 1744034966}

In [26]:
from order.price_agent import pricing_agent
from semantic_kernel.contents import ChatHistory

h = ChatHistory()
h.add_user_message("What is the price for SKU-A100?")

res = await pricing_agent.get_response(h)

ServiceResponseException: ("<class 'semantic_kernel.connectors.ai.open_ai.services.azure_chat_completion.AzureChatCompletion'> service failed to complete the prompt", AuthenticationError("Error code: 401 - {'statusCode': 401, 'message': 'Unauthorized. Access token is missing, invalid, audience is incorrect (https://cognitiveservices.azure.com), or have expired.'}"))

In [ ]:
res.content

'The pricing details for SKU-A100 are as follows:\n\n- **Standard Unit Price**: $10.00\n- **Quantity Ordered**: 1\n- **Final Unit Price (after discounts)**: $10.00\n- **Line Total**: $10.00\n\n### Pricing Breakdown:\n- **Standard Pricing Applied**: $10.00\n- **Customer-Specific Pricing**: Not applicable\n- **Quantity Discount**: Not applicable\n- **Discount Applied**: 0%\n- **Final Price Calculation**: 1 unit × $10.00 = $10.00\n\nThe final total for SKU-A100 is **$10.00**.'

In [5]:
from order.order_team import assistant_team
from semantic_kernel.contents import ChatHistory
from opentelemetry import trace

h = ChatHistory()
h.add_user_message("What is the price for SKU-A100?")


# tracer = trace.get_tracer(__name__)
# with tracer.start_as_current_span("sk_actor._invoke_agent") as span:
res = await assistant_team.get_response(h)

In [6]:
res.content

'Here is the pricing analysis for SKU-A100:\n\n## PRICING ANALYSIS REPORT\nDate: [Current Date]\n\n### EXECUTIVE SUMMARY\n- The listed SKU is SKU-A100.\n- Standard pricing was checked and found no customer-specific price.\n- A quantity discount of 10% was applied to the standard unit price leading to a favorable final unit price.\n- Final order value for SKU-A100 totals $7,200.00.\n\n### DETAILED LINE-ITEM ANALYSIS\n\n#### SKU: SKU-A100\n- Standard Unit Price: $10.00\n- Quantity: 800\n- Line Subtotal (Standard): $8,000.00\n\n##### Pricing Adjustments:\n- Customer-Specific Price: $[Not Applicable]\n- Quantity Discount: 10% [APPLIED]\n- Savings from Standard: $800.00 (Standard price $8,000.00 - Final price $7,200.00)\n\n##### Final Pricing:\n- Final Unit Price: $9.00 \n- Line Total: $7,200.00\n- Pricing Rationale: A quantity discount of 10% was applied to the standard unit price resulting in a final unit price of $9.00.\n\n### ORDER TOTALS\n- Subtotal (Before Discounts): $8,000.00\n- Tot